In [1]:
import java.io.File

val currentDir = System.getProperty("user.dir")
val mainFolder = File(currentDir, "DoomRPG")
mainFolder.mkdir()
val tempFolder = File(mainFolder, "Temporary")
tempFolder.mkdir()
val filesFolder = File(tempFolder, "Files")
filesFolder.mkdir()
val brewBinaryFolder = File(tempFolder, "Brew Binary")
brewBinaryFolder.mkdir()
val desktopBinaryFolder = File(tempFolder, "Desktop Binary")
desktopBinaryFolder.mkdir()

true